In [ ]:
from pandas import DataFrame, read_csv
import matplotlib.pyplot as plt
import pandas as pd
from numpy import math

In [ ]:
file = r"us-counties.csv"
df = pd.read_csv(file, parse_dates=["date"])
df["cfr"] = df["deaths"]/df["cases"]
df["county_state"] = df["county"]+", "+df["state"]

state_names = df["state"].unique()
df = df.drop(columns=["county", "state", "fips"])

state_names

In [ ]:
last_updated = df["date"].max()
print("Last updated:", last_updated)
print("Time since first case:", df["date"].max() - df["date"].min())

In [ ]:
today = df.loc[df["date"] == last_updated, [
    "county_state", "cases", "deaths", "cfr"]]

def get_top(df, column, num_to_show=5):
    if column == "cfr":
        df = df.dropna()
    df = df[df["cfr"] <= 1]
    df = df.sort_values(by=column)
    lo = df.head(num_to_show)
    hi = df.tail(num_to_show).sort_values(by=column, ascending=False)
    return lo, hi

lo, hi = get_top(today, "deaths")
hi

In [ ]:
diff_1_suffix = " - Daily Increase"
diff_2_suffix = " - Change in Daily Increase"

window_size = 5
counties = df.pivot(index="date", columns="county_state", values="cases")
for county in counties.columns:
    counties[county+diff_1_suffix] = counties[county].diff().rolling(window_size, center=True).mean()
    counties[county+diff_2_suffix] = counties[county+diff_1_suffix].diff()
    
counties

In [ ]:
import os

for state in state_names:
    county_names = [c for c in counties.columns if c.endswith(state)]

    os.makedirs(f'plots/{state}', exist_ok=True)
    plt.rcParams['figure.figsize'] = [15, 10]

    for county in county_names:
        df = counties[[county, county+diff_1_suffix, county+diff_2_suffix]
                 ].dropna()
        if not df.empty:
            df.plot.bar(subplots=True)
            plt.savefig(f'plots/{state}/{county}.png', bbox_inches='tight')
            plt.close()

In [ ]:
daily_increase = counties[[county+diff_1_suffix]]
weekday = daily_increase.groupby([daily_increase.index.weekday]).sum()

plt.rcParams['figure.figsize'] = [15, 5]
weekday.plot.bar()